In [1]:
'''
Baseline with RBF kernel C-support SVM
'''

'\nBaseline with RBF kernel C-support SVM\n'

In [1]:
# Import libraries
from linear_classifier import *
from linear_svm import *
import time
import os
import numpy as np 
import json
import imghdr
from PIL import Image
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog, daisy
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score

In [4]:
def load_image( infilename ) :
    '''
    Helper function to manually load in images. Accepts image path infilename 
    and returns the numpy array representation of the image data
    (Need final X to be of dims (N, H, W, D))
    '''
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

def loadData(rootDir, featureType):
    '''
    Helper function which goes into one of our split dataset folders (train, val, test) to extract
    image data. Loads images, performs custom feature extraction, extracts class label from file name,
    and returns two numpy arrays X and y with data / labels for all examples in rootDir
    '''
    featList = []
    labelList = []
    imgList = []
    ppc = 8
    for c in os.listdir(rootDir):
        for file in os.listdir(os.path.join(rootDir, c)):
            filePath = os.path.join(rootDir, c, file)
            if imghdr.what(filePath):
                imRaw = load_image(filePath)
                if featureType == "hog":
                    feature = hog(imRaw, orientations=8, pixels_per_cell=(ppc,ppc), block_norm = "L2",
                                    feature_vector = True, visualize = False, cells_per_block=(4,4)) # HOG Features
                elif featureType == "daisy":
                    imGray = color.rgb2gray(imRaw)
                    feature = daisy(imGray, step=4, radius=15, rings=3, histograms=8, orientations=8,
                                    normalization="l1", visualize = False) # DAISY Features (similar to SIFT)
                # Note: dataset-split/val\compost\compost168.jpg may not be resized yet!
                else:
                    print("Invalid custom feature type specified")
                    exit(0)
                if imRaw.shape != (384, 512, 3):
                    print("Skipping dimension-incompatible image {}".format(file))
                    continue
                featList.append(feature)
                labelList.append(c)
                
    X = np.stack(tuple(featList))
    y = np.array(labelList)
    return X, y

In [5]:
'''
Load in the training and validation data and perform HOG feature extraction.
'''
X_train, y_train = loadData("../../datasets/trashnet/data/dataset-split/train", "hog")
X_val, y_val = loadData("../../datasets/trashnet/data/dataset-split/val", "hog")


In [6]:
# Sanity check dimensions
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(1574, 351360)
(1574,)
(788, 351360)
(788,)


In [7]:
# Preprocessing: subtract the mean image
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(float) - mean_image
X_val = X_val.astype(float) - mean_image

In [ ]:
# Append the bias dimension of ones (i.e. bias trick) so that our SVM
# only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])

In [ ]:
# Encode the class labels
from sklearn.preprocessing import LabelEncoder
classes = ["cardboard", "compost", "glass", "metal", "paper", "plastic", "trash"]
le = LabelEncoder()
le.fit(classes)
y_train = le.transform(y_train)
y_val = le.transform(y_val)

In [ ]:
# Train the SVC model
clf = SVC(kernel="rbf", gamma=0.5, verbose=True, C = 0.1)
clf.fit(X_train, y_train)

[LibSVM]

In [14]:
# Evaluate validation set acc
y_val_pred = clf.predict(X_val)
val_accuracy = np.mean(y_val_pred == y_val)
print('Baseline validation set accuracy is %f' % val_accuracy)

Baseline validation set accuracy is 0.215736


In [15]:
# Training set accuracy
y_train_pred = clf.predict(X_train)
train_accuracy = np.mean(y_train == y_train_pred)
print('Baseline train set accuracy is %f' % train_accuracy)

Baseline train set accuracy is 0.998094
